# Code Clause

Sakshi Shete - Data Science Intern

Task1 - Fake News Detection

Fake News is false information and hoaxes spread through social media and other online media to achieve a political agenda.t is necessary to detect fake news mainly for media outlets to have the ability to attract viewers to their website to generate online advertising revenue. Here I have used python to build a model that can accurately detect whether a piece of news is real or fake.

Link for Dataset: https://drive.google.com/file/d/1er9NJTLUA3qnRuyhfzuN0XUsoIC4a-_q/view

In [1]:
import pandas as pd
import numpy as np
import json 
import random


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

#Reading the data
data = pd.read_csv(r'C:\Users\Acer\Downloads\news\news.csv')
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [2]:
#Dropping the unwanted Column
data = data.drop(['Unnamed: 0'],axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
#Encoding the labels
la = preprocessing.LabelEncoder()
la.fit(data['label'])
data['label'] = la.transform(data['label'])

In [4]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [5]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

In [11]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(sequences1, padding=padding_type, truncating= trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [12]:
embeddings_index = {}



# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embeddings_matrix[i] = embedding_vector

In [13]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,input_length=max_length, weights=[embeddings_matrix],trainable=False),tf.keras.layers.Dropout(0.2),tf.keras.layers.Conv1D(64, 5, activation='relu'),tf.keras.layers.MaxPooling1D(pool_size=4),tf.keras.layers.LSTM(64),tf.keras.layers.Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 54, 50)            377600    
                                                                 
 dropout_1 (Dropout)         (None, 54, 50)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 64)            16064     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 12, 64)           0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                      

In [14]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,epochs=num_epochs,validation_data=(testing_padded,testing_labels),verbose=2)

Train on 2700 samples, validate on 300 samples
Epoch 1/50


C:\Users\Acer\anaconda3\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 3s - loss: 0.6932 - accuracy: 0.5063 - val_loss: 0.6937 - val_accuracy: 0.4800 - 3s/epoch - 1ms/sample
Epoch 2/50
2700/2700 - 1s - loss: 0.6933 - accuracy: 0.4959 - val_loss: 0.6936 - val_accuracy: 0.4800 - 807ms/epoch - 299us/sample
Epoch 3/50
2700/2700 - 1s - loss: 0.6932 - accuracy: 0.5063 - val_loss: 0.6935 - val_accuracy: 0.4800 - 747ms/epoch - 277us/sample
Epoch 4/50
2700/2700 - 1s - loss: 0.6933 - accuracy: 0.5063 - val_loss: 0.6936 - val_accuracy: 0.4800 - 755ms/epoch - 280us/sample
Epoch 5/50
2700/2700 - 1s - loss: 0.6932 - accuracy: 0.5063 - val_loss: 0.6935 - val_accuracy: 0.4800 - 812ms/epoch - 301us/sample
Epoch 6/50
2700/2700 - 1s - loss: 0.6932 - accuracy: 0.5063 - val_loss: 0.6938 - val_accuracy: 0.4800 - 804ms/epoch - 298us/sample
Epoch 7/50
2700/2700 - 1s - loss: 0.6932 - accuracy: 0.5063 - val_loss: 0.6937 - val_accuracy: 0.4800 - 789ms/epoch - 292us/sample
Epoch 8/50
2700/2700 - 1s - loss: 0.6932 - accuracy: 0.5063 - val_loss: 0.6936 - val_accuracy: 0.48

In [21]:
# sample text to check if fake or not
X = "You Can Smell Hillary’s Fear"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,padding=padding_type,truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
	print("This news is Real")
else:
	print("This news is Fake")

This news is Fake


In [24]:
# sample text to check if fake or not
X = "Kerry to go to Paris in gesture of sympathy"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,padding=padding_type,truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] <= 0.5):
	print("This news is Real")
else:
	print("This news is Fake")

This news is Real
